In [1]:
import requests
import pickle
import json
import pyppeteer
from tqdm import *
import re

In [10]:
API_KEY = 'xxx'
API_KEY_SECRET = 'xxx'
BEARER_TOKEN = 'xxx'

In [3]:
headers = {'Authorization': f'Bearer {BEARER_TOKEN}'}

In [4]:
requests.get('https://api.twitter.com/2/users/by/username/EichlerMaor', headers=headers).json()

{'data': {'id': '4004052706',
  'name': 'maor eichler',
  'username': 'EichlerMaor'}}

In [12]:
results = []
records = []
while True:
    print(len(records))
    pagination_token = results[-1]['meta']['next_token'] if results else None
    # https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets
    r = requests.get('https://api.twitter.com/2/users/4004052706/tweets', params={
        'tweet.fields': 'created_at',
        'exclude': 'retweets',
        'max_results': 100,
        'pagination_token': pagination_token,
    }, headers=headers)
    r.raise_for_status()
    d = r.json()
    results.append(d)
    records.extend(d['data'])
    if 'next_token' not in d['meta']:
        break
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [14]:
records[-2]

{'created_at': '2021-06-13T16:59:48.000Z',
 'id': '1404121335564062722',
 'text': '@noahroth Ah, but can you prove it?'}

In [18]:
with open('/home/noam/Downloads/maor.json', 'w') as f:
    json.dump(records, f, indent=2)

In [33]:
# Based on https://github.com/ozgrozer/tweet-image/blob/master/src/backend/createScreenshot.js
tweet_id = 1404121335564062722
async def create_screenshot(fn, page, tweet_id):
    width = 1000
    padding = 25

    await page.goto(
        f'https://platform.twitter.com/embed/index.html?dnt=true&embedId=twitter-widget-0&'
        f'frame=false&hideCard=false&hideThread=false&id={tweet_id}&lang=en&theme=light&'
        f'widgetsVersion=ed20a2b%3A1601588405575',
        waitUntil='networkidle0')

    embedDefaultWidth = 550
    percent = width / embedDefaultWidth
    pageWidth = int(round(embedDefaultWidth * percent))
    pageHeight = 100
    await page.setViewport(dict(width=pageWidth, height=pageHeight))

    await page.evaluate(f"""
    const style = document.createElement('style')
    style.innerHTML = "* {{ font-family: -apple-system, BlinkMacSystemFont, Ubuntu, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol' !important; }}"
    document.getElementsByTagName('head')[0].appendChild(style)

    const body = document.querySelector('body')
    body.style.padding = `{padding}px`
    //body.style.backgroundColor = theme === 'dark' ? '#000' : '#fff'
    body.style.zoom = `{100 * percent}%`
    const articleWrapper = document.querySelector('#app > div')
    articleWrapper.style.border = 'none'
    """)
    img = await page.screenshot(path=fn, fullPage=True)

In [31]:
browser = await pyppeteer.launch()
page = await browser.newPage()

In [37]:
errors = []
for record in tqdm(records):
    tweet_id = record['id']
    try:
        await create_screenshot(f'/home/noam/Downloads/maor/screenshots/{tweet_id}.png', page, tweet_id)
    except Exception as e:
        print(e)
        errors.append(tweet_id)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3161/3161 [2:04:25<00:00,  2.36s/it]


In [40]:
tweet_id = 1404121335564062722
await page.goto(
    f'https://platform.twitter.com/embed/index.html?dnt=true&embedId=twitter-widget-0&'
    f'frame=false&hideCard=false&hideThread=false&id={tweet_id}&lang=en&theme=light&'
    f'widgetsVersion=ed20a2b%3A1601588405575',
    waitUntil='networkidle0')
await page.setViewport(dict(width=550, height=100))
img = await page.screenshot(path='/tmp/img.png', fullPage=True)

In [5]:
all_tweets_txt = open('/home/noam/Downloads/maor/EichlerMaor.txt').read()

In [6]:
all_ids = re.findall(r'EichlerMaor/status/(\d+)', all_tweets_txt)

In [7]:
len(all_ids)

12027

In [8]:
async def create_screenshot2(fn, page, tweet_id):
    await page.goto(
        f'https://platform.twitter.com/embed/index.html?dnt=true&embedId=twitter-widget-0&'
        f'frame=false&hideCard=false&hideThread=false&id={tweet_id}&lang=en&theme=light&'
        f'widgetsVersion=ed20a2b%3A1601588405575',
        waitUntil='networkidle0')
    await page.setViewport(dict(width=550, height=100))
    img = await page.screenshot(path=fn, fullPage=True)

In [9]:
browser = await pyppeteer.launch()
page = await browser.newPage()
errors = []
for tweet_id in tqdm(all_ids):
    try:
        await create_screenshot2(f'/home/noam/Downloads/maor/shots/{tweet_id}.png', page, tweet_id)
    except Exception as e:
        print(e)
        errors.append(tweet_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 12027/12027 [7:18:54<00:00,  2.19s/it]
